In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-darkgrid')

C:\Users\Victo\AppData\Local\Temp\ipykernel_17768\3818186681.py:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


In [2]:
df = pd.read_csv('data_cleaned.csv')

In [3]:
# Import the function
from sklearn.model_selection import train_test_split

# Split of feaures and outcomes
X = df.drop(['outcome_profit'],1)
y = df['outcome_profit']

# Perform train/test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

C:\Users\Victo\AppData\Local\Temp\ipykernel_17768\797845231.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df.drop(['income_am'],1)


In [4]:
from sklearn.preprocessing import StandardScaler

num_feat = X_train.select_dtypes(include=['int64', 'float64']).columns

scaler = StandardScaler()
scaler.fit(X_train[num_feat])

X_train_stan = X_train.copy()

X_train_stan[num_feat] = scaler.transform(X_train[num_feat])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4947 entries, 0 to 4946
Data columns (total 100 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   income_am               4947 non-null   float64
 1   profit_last_am          4947 non-null   float64
 2   profit_am               4947 non-null   float64
 3   damage_am               4947 non-null   float64
 4   crd_lim_rec             4947 non-null   float64
 5   cab_requests            4947 non-null   float64
 6   bar_no                  4947 non-null   float64
 7   neighbor_income         4947 non-null   float64
 8   age                     4947 non-null   float64
 9   tenure_mts              4947 non-null   float64
 10  tenure_yrs              4947 non-null   float64
 11  claims_am               4947 non-null   float64
 12  nights_booked           4947 non-null   float64
 13  shop_am                 4947 non-null   float64
 14  outcome_profit          4947 non-null  

# Linear Regression

In [6]:
from sklearn.linear_model import LinearRegression
reg= LinearRegression()
reg.fit(X_train_stan, y_train)

LinearRegression()

In [7]:
predictions_train = reg.predict(X_train_stan)

coefficient of determination

In [8]:
from sklearn.metrics import r2_score
print(r2_score(y_train, predictions_train))

0.7609736405752742


MAE

In [9]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_train, predictions_train))

1556.931499991138


MSE

In [10]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_train, predictions_train))

18047231.33533516


# Polynomial Regression

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score

avg_scores = [None] * 5

for i in np.arange(1,6):
    
    reg_poly = Pipeline([('poly', PolynomialFeatures(degree=i)),
                  ('linear', LinearRegression(fit_intercept=False))])
    
    scores = cross_val_score(reg_poly, X_train_stan, y_train, scoring = 'r2', cv=3)
    
    avg_scores[i-1] = scores.mean()
    
    print("Order "+str(i)+": avg R^2 = "+str( avg_scores[i-1]))

Order 1: avg R^2 = -8.34956103560475e+24
Order 2: avg R^2 = -25.926666393856987


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
plt.scatter(np.arange(1,6), avg_scores, c='b', label='data')
plt.axis('tight')
plt.title("Cross-validation polynomials")
ax.set_xlabel("Order");
ax.set_ylabel("CV R^2");
plt.tight_layout()
plt.show()

# KNN

# Tree based